In [11]:
import polars as pl
import pandas as pd

import pandas_profiling as pf
pl.Config.set_fmt_str_lengths(50)

swift_does_file='https://drive.google.com/u/0/uc?id=1AxOZbkyk3goruw8-y9MdjalekwEJyTrf&export=download'
transaction_file='https://drive.google.com/u/0/uc?id=14Vth2qaJPj_Iq0JyC-3aMo9-Jfa9wPXg&export=download'

df_swift_codes=pl.from_pandas(pd.read_csv(swift_does_file))
df_transactions=pl.from_pandas(pd.read_csv(transaction_file))

pf.ProfileReport(pd.read_csv(swift_does_file)).to_file('switft_profile.html')
df_swift_codes.head(5)



Export report to file: 100%|██████████| 1/1 [00:00<00:00, 379.16it/s]


Bank,SWIFT code,Check Digits
str,str,str
"""Lloyds Bank""","""LOYD""","""C1"""
"""Barclays Bank""","""BARC""","""22"""
"""Halifax""","""HLFX""","""22"""
"""HSBC""","""HBUK""","""4B"""
"""Natwest""","""NWBK""","""2L"""


In [13]:


pf.ProfileReport(pd.read_csv(transaction_file)).to_file('transactions_profile.html')
df_transactions.head(5)


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 1108.72it/s]


Transaction ID,Account Number,Sort Code,Bank
i64,i64,str,str
3888,62230725,"""95-98-82""","""Data Source Bank"""
4746,83172326,"""42-86-38""","""Barclays Bank"""
5404,34302539,"""53-28-21""","""Barclays Bank"""
9013,13350031,"""93-87-71""","""Natwest"""
2535,68745993,"""57-14-32""","""Barclays Bank"""


In [3]:
df_transactions=df_transactions.with_columns(
    pl.col('Sort Code').str.splitn('-',3).alias('fields')).unnest('fields')


df_transactions=df_transactions.with_columns(
    (pl.col('field_0') + pl.col('field_1') + pl.col('field_2')).alias('new_Sort Code')).drop(['field_0','field_1','field_2','Sort Code'])

df_transactions.head(3)

Transaction ID,Account Number,Bank,new_Sort Code
i64,i64,str,str
3888,62230725,"""Data Source Bank""","""959882"""
4746,83172326,"""Barclays Bank""","""428638"""
5404,34302539,"""Barclays Bank""","""532821"""


In [4]:
df_transactions=df_transactions.join(
    df_swift_codes,
    left_on='Bank',
    right_on='Bank',
    how='left'

)

df_transactions.head(3)

Transaction ID,Account Number,Bank,new_Sort Code,SWIFT code,Check Digits
i64,i64,str,str,str,str
3888,62230725,"""Data Source Bank""","""959882""","""DSBX""","""12"""
4746,83172326,"""Barclays Bank""","""428638""","""BARC""","""22"""
5404,34302539,"""Barclays Bank""","""532821""","""BARC""","""22"""


In [5]:
df_transactions=df_transactions.with_columns(
    pl.lit('GB').alias('Country Code')
)

df_transactions.head(3)

Transaction ID,Account Number,Bank,new_Sort Code,SWIFT code,Check Digits,Country Code
i64,i64,str,str,str,str,str
3888,62230725,"""Data Source Bank""","""959882""","""DSBX""","""12""","""GB"""
4746,83172326,"""Barclays Bank""","""428638""","""BARC""","""22""","""GB"""
5404,34302539,"""Barclays Bank""","""532821""","""BARC""","""22""","""GB"""


In [6]:
df_transactions=df_transactions.with_columns(
    pl.col('Account Number').cast(pl.Utf8)
)

In [7]:
df_transactions=df_transactions.with_columns(
    (pl.col('Country Code') + pl.col('Check Digits') + pl.col('new_Sort Code') +  pl.col('Account Number')).alias('IBAN')
)

In [8]:
df_transactions=df_transactions.select(pl.col('Transaction ID','IBAN'))

df_transactions.write_csv('output.csv')